# timm（PyTorch Image Models）
迁移学习模块，这是目前最强大的 `PyTorch `视觉模型库之一，提供了大量 预训练好的模型（CNN、ViT、Swin、ConvNeXt 等），非常适合做迁移学习。

## 一、`timm` 简介

功能：提供大量 `SOTA` 图像分类、检测、分割模型，带预训练权重。

- 优势：
    
    - 模型数量非常多（700+）
    
    - 支持 `ImageNet-1K`、`ImageNet-21K` 等预训练
    
    - API 统一，方便调用
 
---

## 二、timm 的基本用法
### 1. 查看可用模型

In [ ]:
import timm

# 列出所有模型
print(timm.list_models())

# 按关键字搜索，例如只看 resnet 系列
print(timm.list_models('*resnet*'))


### 验证是否支持动态尺寸
- 可以通过以下代码测试：

In [ ]:
import torch

model = timm.create_model('efficientnet_b0', pretrained=True)
dummy_input = torch.randn(1, 3, 32, 32)  # 尝试 32×32 输入
try:
    output = model(dummy_input)
    print("模型支持动态尺寸！输出形状：", output.shape)
except RuntimeError as e:
    print("报错：", e)

### 2. 创建模型

In [ ]:
import timm
import torch.nn as nn

# 创建一个预训练的 resnet50
model = timm.create_model('resnet50', pretrained=True)

# 替换最后分类层（默认 1000 类 → 修改为 10 类）
num_classes = 10
model.fc = nn.Linear(model.fc.in_features, num_classes)


### 3. 迁移学习（特征提取 vs 微调）

- 特征提取：冻结主干网络参数，只训练最后一层

In [ ]:
for param in model.parameters():
    param.requires_grad = False   # 作用是冻结模型的所有参数​（即关闭梯度计算）


- 部分冻结​：若只需冻结部分层，可通过筛选参数实现

In [ ]:
for name, param in model.named_parameters():
    if 'layer1' in name: # 只冻结特定层
        param.requires_grad = False

### 注意事项
​验证模式​：冻结参数后，通常需调用 `model.eval()`关闭`BatchNorm`和`Dropout`的随机性。

---
- 微调：允许全部参数训练（更耗时）

In [ ]:
for param in model.parameters():
    param.requires_grad = True
    

---
## 三、timm.create_model 常见参数

```python
timm.create_model(
    model_name,               # 模型名称（必须参数）
    pretrained=False,         # 是否加载预训练权重
    num_classes=1000,         # 分类类别数
    in_chans=3,              # 输入通道数
    checkpoint_path='',      # 自定义权重路径
    scriptable=None,         # 是否可脚本化
    exportable=None,         # 是否可导出
    pretrained_cfg=None,     # 预训练配置
    **kwargs                 # 其他关键字参数
)
```

| 参数                | 作用                                                                               |
| ----------------- | -------------------------------------------------------------------------------- |
| `model_name`      | 模型名称，例如 `"resnet50"`, `"vit_base_patch16_224"`, `"swin_base_patch4_window7_224"` |
| `pretrained`      | 是否加载预训练权重（`True/False`）                                                          |
| `num_classes`     | 输出类别数（默认 1000，需改为你的任务类别数）                                                        |
| `in_chans`        | 输入通道数（默认 3，可改成 1 适应灰度图）                                                          |
| `checkpoint_path` | 自定义权重路径                                                                          |
| `scriptable`      | 是否兼容 TorchScript                                                                 |
| `exportable`      | 是否兼容 ONNX 导出                                                                     |
| `pretrained_cfg`  | 选择具体的预训练配置（例如 imagenet-1k / 21k）                                                 |


---

## 四、迁移学习完整示例

以 ViT (Vision Transformer) 为例：

In [ ]:
import timm
import torch
import torch.nn as nn
import torch.optim as optim

# 1. 加载预训练模型
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=10)

# 2. 迁移学习：只训练分类头
for param in model.parameters():
    param.requires_grad = False
for param in model.head.parameters():
    param.requires_grad = True

# 3. 损失函数 & 优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.head.parameters(), lr=1e-3)

# 4. 前向传播测试
x = torch.randn(4, 3, 224, 224)  # batch=4, 输入224x224
y = model(x)
print(y.shape)  # torch.Size([4, 10])


## 五、常见技巧

- 1.快速更换模型：只要改 `model_name` 就能切换 `ResNet`、`EfficientNet`、`ViT` 等。

- 2.自定义输入通道：例如医学影像是单通道灰度图 → `in_chans`=1。

- 3.大规模预训练模型：很多模型支持 `imagenet21k`，适合小数据集迁移学习。

- 4.冻结/解冻策略：

    - 前期只训练分类头 → 收敛快。
    
    - 后期解冻部分 `backbone` → 提升性能。

---
## 六、总结

- timm.create_model() 是入口，核心参数：model_name、pretrained、num_classes、in_chans。

- 小数据集 → 特征提取（冻结 backbone）。

- 大数据集 → 微调（解冻 backbone）。

- `timm` 的优势是 模型库丰富，基本涵盖所有主流 CNN & Transformer。

# 使用 timm 做特征提取

## 1. timm 加载模型（去掉分类头）

In [ ]:
import timm
import torch
import torch.nn as nn

# 以 ViT 为例
model = timm.create_model('vit_base_patch16_224', pretrained=True)

# 查看模型结构
print(model)


- 如果只想要特征，不要分类头，可以直接：

In [ ]:
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=0) # 输出改为 0 


- 这样 `model(x)`· 的输出就是 特征向量（默认是 `global pooled feature`）。

## 2. 获取特征维度

In [ ]:
dummy = torch.randn(1, 3, 224, 224)  # 假输入
features = model(dummy)
print(features.shape)  # [1, feature_dim]

# 比如 vit_base_patch16_224 输出 [1, 768] 特征。

## 3. 接上自定义分类器

- 如果你要在提取的特征上加自己的分类头：

In [ ]:
class CustomNet(nn.Module):
    def __init__(self, backbone_name='vit_base_patch16_224', num_classes=10):
        super().__init__()
        # 去掉最后分类头
        self.backbone = timm.create_model(backbone_name, pretrained=True, num_classes=0)
        in_features = self.backbone.num_features  # 特征维度

        # 自定义分类器
        self.classifier = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes),
            # nn.Softmax(dim=1)   # 如果做分类  分类任务里，通常 不写 Softmax，而是直接输出 logits，然后配合 nn.CrossEntropyLoss
        )

    def forward(self, x):
        feats = self.backbone(x)      # 提取特征
        out = self.classifier(feats)  # 分类
        return out


## 4. 提取中间层特征

- 有时不想只要最终 `pooled feature`，可以用 `forward_features`：

In [ ]:
backbone = timm.create_model('resnet50', pretrained=True)

x = torch.randn(1, 3, 224, 224)
feats = backbone.forward_features(x)  # 中间层特征
print(feats.shape)  # e.g. [1, 2048, 7, 7]


- 这样你能拿到卷积特征图，再自己做 `pooling` 或`attention`。
- `pooling（池化）`：像“缩小图片”，让特征变小，但仍保留关键信息。
- `attention（注意力机制）`：像“聚光灯”，让模型自动决定该重点关注哪些部分。

## 5. 冻结特征提取器

- 如果你只想用预训练特征，不训练 `backbone`，可以：

In [ ]:
for param in model.backbone.parameters():
    param.requires_grad = False


In [ ]:
- 这样训练时只更新你定义的 `classifier`

# ⚡ 总结：

- `num_classes`=0 → 去掉分类头，直接提取特征。

- `forward_features(x)` → 提取中间层特征图。

- 接上 `nn.Sequential` 就能替换为你自己的分类器